In [1]:
import numpy as np
from pyscf import gto, scf, fci

In [2]:
# for spinorbitals
def ONERDMFT_hartree_energy(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 0.5*energy

In [3]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [4]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #
#mol.basis = "ccpvdz"
#, mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

print(eri.shape)

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()

N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)

occu, naturalC = np.linalg.eigh(gamma)

(2, 2, 2, 2)
converged SCF energy = -2.85516042615445


In [5]:
#print(eri)

In [6]:
PE = expand_matrix(P)
print(PE)

[[0.35056022 0.30408465 0.         0.        ]
 [0.30408465 0.26377059 0.         0.        ]
 [0.         0.         0.35056022 0.30408465]
 [0.         0.         0.30408465 0.26377059]]


In [7]:
CE = expand_matrix(C)
print(CE)

[[ 0.59208126 -1.14981805  0.          0.        ]
 [ 0.51358601  1.1869588   0.          0.        ]
 [ 0.          0.          0.59208126 -1.14981805]
 [ 0.          0.          0.51358601  1.1869588 ]]


In [8]:
SE = expand_matrix(S)
print(SE)

[[1.         0.63414774 0.         0.        ]
 [0.63414774 1.         0.         0.        ]
 [0.         0.         1.         0.63414774]
 [0.         0.         0.63414774 1.        ]]


In [9]:
gamma = np.matmul(np.matmul(CE.T,np.matmul(np.matmul(SE,PE),SE)), CE)
occu, naturalC = np.linalg.eigh(gamma)

In [10]:
naturalCTT = np.matmul(CE,naturalC)

In [11]:
ONERDMFT_hartree_energy(eri, naturalCTT, occu)

4.107628675501507